IMPORTS

In [ ]:
from main import *
from neural_network import *
import time
import json
from dataset import Dataset
from rise_and_fall import *
from IPython.display import clear_output
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot as plt
import scipy as sci
np.set_printoptions(precision=3, linewidth=2*75, suppress=False)
import os
import glob
import shutil

PLOT FUNTIONS

In [ ]:
def plot_frame(f: pd.DataFrame, mxis=[], mnis=[], mark=None):
    ps = f["meanp"].to_numpy()
    mn = ps.min()
    mx = ps.max()
    plt.plot(ps)
    if len(mxis) > 0:
        plt.vlines(x=mxis, ymin=mn, ymax=mx, colors='green',)
    elif 'mxis' in f.columns:
        plt.vlines(x=f.index[f['mxis'] > 0], ymin=mn, ymax=mx, colors='green',)
    if len(mnis) > 0:
        plt.vlines(x=mnis, ymin=mn, ymax=mx, colors='red',)
    elif 'mnis' in f.columns:
        plt.vlines(x=f.index[f['mnis'] > 0], ymin=mn, ymax=mx, colors='red',)
    if mark is not None:
        plt.vlines(x=[mark], ymin=mn, ymax=mx, colors='blue',)
    plt.show()


def unique(xs):
    ys = []
    for x in xs:
        if x in ys:
            continue
        ys.append(x)
    return ys


In [ ]:
def test(symbol, show=True):
    plt.figure(figsize=(15, 4))
    mtypes = ['ch', 'bs', 'p']
    yps=[]
    yas=[]
    r2s = []
    for mt in mtypes:
        model = get_bp_model(symbol, 3e-4, mt, allow_base=False)
        _,_,data = normalized_data(symbol, mt)
        yp = model.predict(data[:2], batch_size=512, verbose=0)
        yps.extend(yp) if type(yp)==list else yps.append(yp)
        ya = data[2:]
        yas.extend(ya) if type(ya)==list else yas.append(ya)
    
    for i in range(4):
        slope, intercept, r_value, p_value, std_err = sci.stats.linregress(yas[i], yps[i].flatten())
        plt.subplot(1, len(yas), i+1)
        plt.plot(yas[i], slope*yas[i]+intercept, c='r')
        plt.text(0.0, 0.0, f'R2: {np.round(r_value**2, 2)}')
        plt.scatter(yas[i], yps[i])
        r2s.append(r_value**2)
    if show:
        plt.show()
    else:
        return r2s

INDICES CREATION FOR TRAINING VALIDATION AND TESTING

In [ ]:
# with open('database/channels.json') as file:
#     symbols = json.load(file)

# indices = []
# for symbol in symbols:
#     df = pd.read_csv(f"database/{symbol}.csv")
#     limit = df.shape[0]-1000
#     low = 1000
#     s,v = (int(limit*frac) for frac in [5/100, 10/100])
#     r = limit-s-v
#     s_ = np.random.randint(low, limit, size=s)

#     v_ = np.random.randint(low, limit, size=v)
#     similarity = np.isin(v_, s_)
#     while np.any(similarity):
#         v_[similarity] = np.random.randint(low, limit, v_[similarity].shape[0])
#         similarity = np.isin(v_, s_)
    
#     r_ = np.random.randint(low, limit, size=r)
#     similarity = np.isin(r_, np.append(v_, s_))
#     while np.any(similarity):
#         r_[similarity] = np.random.randint(low, limit, r_[similarity].shape[0])
#         similarity = np.isin(r_, np.append(v_, s_))
    
#     indices.append(
#         {
#             'symbol': symbol,
#             'r': r_,
#             's': s_,
#             'v': v_
#         }
#     )

# np.savez_compressed('database/indices.npz', indices=indices)

SAVING DATASETS IN NPY

In [ ]:
with open('database/channels.json') as file:
    symbols = json.load(file)

def save_set(x1, itter, symbol, sc):
    np.save(f'database/buy/{symbol}/{sc}x1.npy', x1)
    x2, y1, y2, y3, y4 = itter
    np.save(f'database/buy/{symbol}/{sc}x2.npy', x2)
    np.save(f'database/buy/{symbol}/{sc}y1.npy', y1)
    np.save(f'database/buy/{symbol}/{sc}y2.npy', y2)
    np.save(f'database/buy/{symbol}/{sc}y3.npy', y3)
    np.save(f'database/buy/{symbol}/{sc}y4.npy', y4)

dataset_size = 350000
trx1 = []; trothers = []
vlx1 = []; vlothers = []
tsx1 = []; tsothers = []

if __name__ == '__main__':
    for symbol in symbols:
        if symbol in [r.split('\\')[-1] for r in glob.glob('database/buy/*') if '.' not in r]:
            continue
        print(symbol, ' started')
        db = Dataset(symbol)
        x1, others = db.get_data_set(
            batch_size=dataset_size, max_procs=4, single_process_size=int(dataset_size/4), buy=True, validation=False, test=False)

        norm = x1.max(axis=(0,1))
        norm[-1] = 288.0
        if not os.path.isdir(f'database/buy/{symbol}'):
            os.mkdir(f'database/buy/{symbol}')
        np.save(f'database/buy/{symbol}/norm.npy', norm)
        save_set(x1, others, symbol, 'tr')

        time.sleep(3*60)

        x1, others = db.get_data_set(
            batch_size=2**12, max_procs=8, single_process_size=512, buy=True, validation=True, test=False)
        save_set(x1, others, symbol, 'vl')

        x1, others = db.get_data_set(
            batch_size=2**12, max_procs=8, single_process_size=512, buy=True, validation=False, test=True)
        save_set(x1, others, symbol, 'ts')

        time.sleep(5*60)
        

In [ ]:
with open('database/channels.json') as file:
    symbols = json.load(file)

def save_set(x1, itter, symbol, sc):
    np.save(f'database/sell/{symbol}/{sc}x1.npy', x1)
    x2, y1, y2 = itter
    np.save(f'database/sell/{symbol}/{sc}x2.npy', x2)
    np.save(f'database/sell/{symbol}/{sc}y1.npy', y1)
    np.save(f'database/sell/{symbol}/{sc}y2.npy', y2)

dataset_size = 64000
trx1 = []; trothers = []
vlx1 = []; vlothers = []
tsx1 = []; tsothers = []

if __name__ == '__main__':
    for symbol in symbols:
        if symbol in [r.split('\\')[-1] for r in glob.glob('database/sell/*') if '.' not in r]:
            continue
        print(symbol, ' started')
        db = Dataset(symbol)
        x1, others = db.get_data_set(
            batch_size=dataset_size, max_procs=4, single_process_size=int(dataset_size/4), buy=False, validation=False, test=False)

        norm = x1.max(axis=(0,1))
        norm[-1] = 288.0
        if not os.path.isdir(f'database/sell/{symbol}'):
            os.mkdir(f'database/sell/{symbol}')
        np.save(f'database/sell/{symbol}/norm.npy', norm)
        save_set(x1, others, symbol, 'tr')

        # time.sleep(3*60)

        x1, others = db.get_data_set(
            batch_size=2**12, max_procs=8, single_process_size=512, buy=False, validation=True, test=False)
        save_set(x1, others, symbol, 'vl')

        x1, others = db.get_data_set(
            batch_size=2**12, max_procs=8, single_process_size=512, buy=False, validation=False, test=True)
        save_set(x1, others, symbol, 'ts')

        time.sleep(3*60)
        

MODEL TESTING

In [ ]:
for symbol in [r.split('\\')[-1] for r in glob.glob('database/buy/*') if '.' not in r]:
    try:
        print(symbol)
        test(symbol)
    except:
        pass

RANDOM

In [ ]:
# model = buy_predictor(finetune=True)
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# model.summary()

In [ ]:
db = Dataset("BTCUSDT")

In [ ]:
def aggregated_minimalistic_frame(db, i, mf, at, kinterval=5.0):
    '''i is the last index of minimalistic frame (index of sell), mf is the minimalistic frame, at is the index for agregation'''
    if mf.shape[0]==0:
        raise Exception('Invalid mf')
    to_be_aggreagted = db.nf.loc[i:at, :]
    vector = np.zeros((mf.shape[1],))
    ft = db.nf.iloc[i: at, :]
    vector[[0, 1]] = ft.loc[ft.index[-1], ['meanp', 'stdp']]
    vector[[3, 5, 6]] = ft.loc[:, ['meanp', 'taker', 'maker']].mean()
    meanpstd = ft.loc[:, 'meanp'].std()
    if np.isnan(meanpstd):
        meanpstd = 0.0
    vector[4] = meanpstd
    vector[[7, 8]] = ft.loc[:, ['vol', 'ntrds']].sum()
    vector[9] = ft.shape[0]*kinterval
    ctime = pd.to_datetime(ft['stime'].iloc[-1], unit='ms')

    a = np.concatenate([mf, vector.reshape(1, -1)], axis=0)
    ctime = ctime.weekday()*288 + (ctime.hour*60.0 + ctime.minute)/5.0
    return a, ctime
    

In [ ]:
at = 80000
max_size=64
kinterval=5.0
a, x, n = db.get_minimalistic_frame(at=at, max_size=64*2)
xnd = []
for i in x:
    xnd.append({
        'index': i,
        'type': 1
    })
for i in n:
    xnd.append({
        'index': i,
        'type': 0
    })
xnf = pd.DataFrame(xnd)
xnf.sort_values('index', ignore_index=True, inplace=True)
xnf = xnf[-max_size*2:].reset_index(drop=True)

inputs=[]
outputs=[]

for i in range(max_size-1, max_size*2-2):
    ji=xnf['index'][i]+1
    jf=xnf['index'][i+1]
    j=np.random.randint(ji, jf+1)
    ins = aggregated_minimalistic_frame(db, xnf['index'][i], a[(i-max_size+1):i], j); inputs.append(ins)
    if xnf['type'][i]==0:
        #'0 means i is mni' mean its bullish
        buy = (xnf['index'][i+2]-j)*kinterval
        sell = (xnf['index'][i+1]-j)*kinterval
        #last and future prices are correct think before change
        future_price = db.nf.loc[xnf['index'][i+1], 'meanp']
        last_price = db.nf.loc[xnf['index'][i], 'meanp']
    else:
        #'1 means i is mxi' mean its bearish
        buy = (xnf['index'][i+1]-j)*kinterval
        sell = (xnf['index'][i+2]-j)*kinterval
        #last and future prices are correct think before change
        future_price = db.nf.loc[xnf['index'][i+2], 'meanp']
        last_price = db.nf.loc[xnf['index'][i+1], 'meanp']

    ch = (future_price - last_price)/last_price
    outputs.append([buy, future_price, sell, ch*100.0])


In [ ]:
trans='sell'
for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    for mtype in ['s', 'p']:
        # reset_history(symbol=symbol, mtype=mtype, trans=trans, new_name="SBASE")
        with open(f'database/{trans}/{symbol}/{mtype}_history/version_history.json', 'r') as file:
            d = json.load(file)

        d[-1]["id"] = "SBASE"
        with open(f'database/{trans}/{symbol}/{mtype}_history/version_history.json', 'w') as file:
            json.dump(d, file)

In [ ]:
trans='buy'
for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    for mtype in ['ch', 'b', 's', 'p']:
        reset_history(symbol=symbol, mtype=mtype, trans=trans, new_name="Base")

In [ ]:
trans='sell'
for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    for mtype in ['s', 'p']:
        reset_history(symbol=symbol, mtype=mtype, trans=trans, new_name="Base")

In [ ]:
temp_dat={
    "id": "Base",
    "time": 1672842686.6763935,
    "loss": 0.094559235792411,
    "y1_loss": 0.04415296723968104,
    "y3_loss": 0.050406275022971,
    "val_loss": 0.09608950583558333,
    "val_y1_loss": 0.04403252115375117,
    "val_y3_loss": 0.05205698625037545,
    "lr": 1,
    "epochs": 19
}

for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    trans='buy'
    mtype = 'bs'
    model = get_bp_model(symbol=symbol, lr=3e-4, mtype=mtype)

    bmodel = b_predictor()
    smodel = s_predictor()
    for i in range(5):
        smodel.layers[i].set_weights(model.layers[i].weights)
    smodel.layers[5].set_weights(model.layers[6].weights)
    smodel.layers[6].set_weights(model.layers[5].weights)
    for i in range(7,10):
        smodel.layers[i].set_weights(model.layers[i].weights)
    smodel.layers[10].set_weights(model.layers[11].weights)
    smodel.layers[11].set_weights(model.layers[10].weights)
    for i in range(12,len(model.layers)):
        smodel.layers[i].set_weights(model.layers[i].weights)

    for i in range(11):
        bmodel.layers[i].set_weights(model.layers[i].weights)

    os.mkdir(f"database/{trans}/{symbol}/b_history")
    bmodel.save_weights(f"database/{trans}/{symbol}/b_history/Base.h5")
    with open(f"database/{trans}/{symbol}/b_history/version_history.json", 'w') as file:
        json.dump([temp_dat], file)
    os.mkdir(f"database/{trans}/{symbol}/s_history")
    smodel.save_weights(f"database/{trans}/{symbol}/s_history/Base.h5")
    with open(f"database/{trans}/{symbol}/s_history/version_history.json", 'w') as file:
        json.dump([temp_dat], file)

for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/buy/*') if '.' not in r]:
    shutil.rmtree(f"database/buy/{symbol}/bs_history")

In [ ]:
for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    for mtype in ['s', 'p']:
        if not os.path.isdir(f"database/sell/{symbol}/{mtype}_history"):
            shutil.copytree(f"database/buy/{symbol}/{mtype}_history", f"database/sell/{symbol}/{mtype}_history")

In [ ]:
sym = "BTCUSDT"
mtype = 's'
trans='buy'
s = normalized_test_data(sym, mtype, trans=trans)
model = get_bp_model(sym, 3e-4, mtype, trans=trans)
with tf.device('/cpu:0'):
    yp = model.predict(s[:2], batch_size=4096).flatten()
plt.scatter(s[2], yp)
slope, intercept, r_value, p_value, std_err = sci.stats.linregress(s[2], yp)
plt.plot(s[2], slope*s[2]+intercept, c='r')
plt.text(0.0, 0.0, f'R2: {np.round(r_value**2, 2)}')

In [ ]:
[r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]

In [ ]:
models = []
sets = []
trans='buy'
for symbol in [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]:
    for mtype in ['ch', 'p', 'b', 's']:
        with tf.device('/cpu:0'):
            model = get_bp_model(symbol, lr=3e-4, mtype=mtype, trans=trans)
            s = normalized_test_data(symbol, mtype, trans=trans)
            models.append(model)
            sets.append(s)

In [ ]:
symbols = [r.split('\\')[-1] for r in glob.glob(f'database/{trans}/*') if '.' not in r]
print("plotting")
j=0
plt.figure(figsize=(4*4, 11*4))
for i in range(0, len(models)):
    # with tf.device('/cpu:0'):
    yp = models[i].predict(sets[i][:2], batch_size=728).flatten()
    plt.subplot(11, 4, i+1)
    plt.scatter(sets[i][2], yp, label=f'{symbols[j]}')
    slope, intercept, r_value, p_value, std_err = sci.stats.linregress(sets[i][2], yp)
    plt.plot(sets[i][2], slope*sets[i][2]+intercept, c='r')
    plt.text(0.0, 0.0, f'R2: {np.round(r_value**2, 2)}')
    plt.legend()
    if i!=0 and (i+1)%4==0:
        j+=1